In [1]:
import json
import numpy as np
import pickle
import pandas as pd
from scipy import stats
from scipy.spatial.distance import pdist
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from classes.behavioural_experiment import Behavioural_experiment
from classes.participant import Participant
from classes.trial import Trial
from classes.models import presets
from methods.extraction import vecToDict, dictToVec
from statannotations.Annotator import Annotator


pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
file_study = open('./data/database.obj','rb')
study = pickle.load(file_study)

In [ ]:
priors_2 = study.priors[study.priors.experiment == 'experiment_2']
priors_2_implausible = priors_2[priors_2.trial_type == 'implausible']
p2_i = priors_2_implausible[priors_2_implausible.columns[3:]].to_numpy()

graph_1 = np.nan_to_num(p2_i[:, :6])
graph_2 = np.nan_to_num(p2_i[:, 6:12])
graph_3 = np.nan_to_num(p2_i[:, 12:])

prior_graphs = graph_1 + graph_2 + graph_3

np.sum(prior_graphs != 0, axis=1).mean()

In [ ]:
posteriors_2 = study.posteriors[study.posteriors.experiment == 'experiment_2']
posteriors_2_implausible = posteriors_2[posteriors_2.trial_type == 'implausible']

p2_i = posteriors_2_implausible[posteriors_2_implausible.columns[3:]].to_numpy()

graph_1 = np.nan_to_num(p2_i[:, :6])
graph_2 = np.nan_to_num(p2_i[:, 6:12])
graph_3 = np.nan_to_num(p2_i[:, 12:])

posterior_graphs = graph_1 + graph_2 + graph_3

np.sum(posterior_graphs != 0, axis=1).mean()

In [ ]:

df = pd.read_csv('./data/df_participants.csv')
sample_size = len(df)
print('Sample size:', sample_size)
experiments = ['experiment_1', 'experiment_2', 'experiment_3', 'experiment_4']
sample_size_exps = [len(df[df.experiment == exp]) for exp in experiments]
print(sample_size_exps)

df_trials = pd.read_csv('./data/df_trials_wprior.csv')

interventions = pd.read_csv('./data/df_interventions.csv')

## Demographics
demos = pd.read_csv('./data/df_demographics.csv')

experiments = df.experiment.unique().tolist()
print(experiments)

df_long_2 = pd.read_csv('./data/accuracy_lf_exp2.csv')
df_long_2_ham = pd.read_csv('./data/editdist_lf_exp2.csv')

df_long_3 = pd.read_csv('./data/accuracy_lf_exp3_wprior.csv')
df_long_3_ham = pd.read_csv('./data/editdist_lf_exp3_wprior.csv')
df_long_3_ham['difficulty'] = df_long_3_ham['difficulty'].replace({'congruent':1.0, 'incongruent':2.0})
#df_long_3['hamming'] = df_long_3_ham.accuracy.to_list()

pids = df_long_3.participant.unique()
diff = ['congruent', 'incongruent']

df_long_3['utid'] = df_long_3.participant + '_' + df_long_3.difficulty.astype(str)
df_long_3_ham['utid'] = df_long_3_ham.participant + '_' + df_long_3_ham.difficulty.astype(str)

df_long_3['hamming'] = np.nan
df_long_3.loc[df_long_3.utid.isin(df_long_3_ham.utid), 'hamming'] = df_long_3_ham.loc[df_long_3_ham.utid.isin(df_long_3.utid), 'accuracy'].to_list()
df_long_3 = df_long_3.drop(['utid'], axis=1)


df_long_4 = pd.read_csv('./data/accuracy_lf_exp4_wprior.csv')
df_long_4_ham = pd.read_csv('./data/editdist_lf_exp4_wprior.csv')
df_long_4_ham['difficulty'] = df_long_4_ham['difficulty'].replace({'congruent':1.0, 'incongruent':2.0})

df_long_4['utid'] = df_long_4.participant + '_' + df_long_4.difficulty.astype(str)
df_long_4_ham['utid'] = df_long_4_ham.participant + '_' + df_long_4_ham.difficulty.astype(str)

df_long_4['hamming'] = np.nan
df_long_4.loc[df_long_4.utid.isin(df_long_4_ham.utid), 'hamming'] = df_long_4_ham.loc[df_long_4_ham.utid.isin(df_long_4.utid), 'accuracy'].to_list()
df_long_4 = df_long_4.drop(['utid'], axis=1)


# Demographics

In [ ]:

selections = [
    ['experiment_1', 'experiment_2', 'experiment_3', 'experiment_4'],
    ['experiment_1'],
    ['experiment_2'],
    ['experiment_3'],
    ['experiment_4'],
]

for experiment in selections:
    demos_local = demos[demos.experiment.isin(experiment)]
    print(f'Selection: {experiment}')
    print(f'Gender: {demos_local[demos_local.gender == 1].shape[0]} males (N = {demos_local.shape[0]})')
    print(f'Age: {demos_local.age.mean()}, sd={demos_local.age.std()}\n')

# Descriptive and general link recovery statistics

In [ ]:
experiments = [
    'experiment_1', 
    'experiment_2', 
    'experiment_3', 
    'experiment_4'
]

for experiment in experiments:
    df_exp = df[df.experiment == experiment]

    print(f'Data for {experiment}')
    # Links recovered
    t, p = stats.ttest_1samp(df_exp.genCorr, 1/5)
    degf = df_exp.genCorr.size - 1
    print(f'Links recovered: {np.round(df_exp.genCorr.mean(), 6)}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}, chance level={1/5}')

    # positive links recovered
    t, p = stats.ttest_1samp(df_exp.posCorr, 2/6)
    degf = df_exp.posCorr.size - 1
    print(f'Positive links recovered: {np.round(df_exp.posCorr.mean(), 6)}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}, chance level={np.round(2/6, 2)}')

    # negative links recovered
    t, p = stats.ttest_1samp(df_exp.negCorr, 2/6)
    degf = df_exp.negCorr.size - 1
    print(f'Negative links recovered: {np.round(df_exp.negCorr.mean(), 6)}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}, chance level={np.round(2/6, 2)}')

    t, p = stats.ttest_ind(df_exp.posCorr, df_exp.negCorr)
    degf = df_exp.posCorr.size - 1 + df_exp.negCorr.size - 1
    print(f'Difference negative positive: diff={df_exp.posCorr.mean() - df_exp.negCorr.mean()}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}')
    
    # weak links recovered
    t, p = stats.ttest_1samp(df_exp.weakCorr, 2/6)
    degf = df_exp.weakCorr.size - 1
    print(f'Weak links recovered: {np.round(df_exp.weakCorr.mean(), 6)}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}, chance level={np.round(2/6, 2)}')

    # strong links recovered
    t, p = stats.ttest_1samp(df_exp.stgCorr, 2/6)
    degf = df_exp.stgCorr.size - 1
    print(f'Strong links recovered: {np.round(df_exp.stgCorr.mean(), 6)}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}, chance level={np.round(2/6, 2)}')

    t, p = stats.ttest_ind(df_exp.weakCorr, df_exp.stgCorr)
    degf = df_exp.weakCorr.size - 1 + df_exp.stgCorr.size - 1
    print(f'Difference weak strong: diff={df_exp.weakCorr.mean() - df_exp.stgCorr.mean()}, t({degf})={np.round(t, 6)}, p={np.round(p, 6)}')
    

    print(f'Average time taken: mean={np.round(df_exp.time_taken.mean(), 6)}, std={np.round(df_exp.time_taken.std(), 6)}')


    # Order in trial
    if experiment[-1] in ['2', '3', '4']:
        if experiment[-1] == '2':
            for i, scenario in enumerate(['finance', 'estate', 'crime']):
                group_1 = df_exp[df_exp[f'{scenario}_cond_order'] == 1][f'{scenario}_acc']
                group_2 = df_exp[df_exp[f'{scenario}_cond_order'] == 2][f'{scenario}_acc']
                group_3 = df_exp[df_exp[f'{scenario}_cond_order'] == 3][f'{scenario}_acc']
            
                f, p = stats.f_oneway(group_1, group_2, group_3)
                print(f'Scenario {scenario} order effect: F={np.round(f, 6)}, p={np.round(p, 6)}')

        if experiment[-1] in ['3', '4']:
            for i, scenario in enumerate(['finance', 'crime']):
                group_1 = df_exp[df_exp[f'{scenario}_cond_order'] == 1][f'{scenario}_acc']
                group_2 = df_exp[df_exp[f'{scenario}_cond_order'] == 2][f'{scenario}_acc']
            
                f, p = stats.f_oneway(group_1, group_2)
                print(f'Scenario {scenario} order effect: F={np.round(f, 6)}, p={np.round(p, 6)}')


    print()


In [ ]:
df_2 = df_trials[df_trials.experiment == 'experiment_2']
df_2

## Experiment 1 percent link recovered and accuracies

### Links Recovered and accuracies (mean and std)

In [ ]:
df_trials1 = df_trials[df_trials.experiment == 'experiment_1']
df_trials1.groupby('trial_name')[['genCorr', 'accuracy']].mean() 

In [ ]:
stats.ttest_ind(df_trials1[df_trials1.trial_name == 'crime'].accuracy, df_trials1[df_trials1.trial_name == 'crime_control'].accuracy)

In [ ]:
df_trials1.groupby('trial_name')[['genCorr', 'accuracy']].std()

### Chain graphs (means and std)

In [ ]:
df_trials1.groupby(['trial_name', 'trial_spec'])[['genCorr', 'accuracy']].mean()

In [ ]:
df_trials1.groupby(['trial_name', 'trial_spec'])[['genCorr', 'accuracy']].std()

In [ ]:
df_trials1.genCorr.mean()

In [ ]:
df_trials1.groupby('trial_name').accuracy.mean()

In [ ]:
df_trials1.groupby('trial_name').genCorr.mean()

In [ ]:
df_trials1.hamming.mean()

In [ ]:
df_trials.columns

## Interventions descriptives

In [ ]:
print('Across experiments')
print('Number of interventions:', len(interventions))
#print('Average number of interventions per participants:', len(interventions) / sample_size)
print('Average number of interventions per trial:', len(interventions) / (sample_size * 4))
print(f'Mean intervention length: {interventions.length_sec.mean()}')
print(f'Std intervention length: {interventions.length_sec.std()}')
print(f'Mean intervening time (in %): {interventions.groupby("utid").relative_length.sum().mean()}')
print(f'Std intervening time (in %): {interventions.groupby("utid").relative_length.sum().std()}')

In [ ]:

for i, experiment in enumerate(experiments):
    interventions_exp = interventions[interventions.experiment == experiment]
    print(f'\n {experiment}')
    #print('Number of interventions:', len(interventions_exp))
    #print('Average number of interventions per participants:', len(interventions_exp) / sample_size_exps[i])
    print('Average number of interventions per trial:', len(interventions_exp) / (sample_size_exps[i] * 4))
    print(f'Mean intervention length: {interventions_exp.length_sec.mean()}')
    print(f'Std intervention length: {interventions_exp.length_sec.std()}')

    print(f'Mean intervening time (in %): {interventions_exp.groupby("utid").relative_length.sum().mean()}')
    print(f'Std intervening time (in %): {interventions_exp.groupby("utid").relative_length.sum().std()}')



## Accuracy Distribution per experiment

In [ ]:
plt.figure(figsize=(12, 5))

for i in range(len(experiments)):
    plt.subplot(1, len(experiments), i+1)
    mean_acc = df[df.experiment == experiments[i]].mean_acc
    sns.histplot(mean_acc, kde=True, stat='density')
    plt.title(f'Mean acc. {experiments[i]}, N = {len(mean_acc)}')
plt.tight_layout()
plt.show()

In [22]:
color_dict = {
    'crime': 'cornflowerblue',
    'finance': 'darkorange',
    'estate': 'seagreen'
}

def gen_colors(labels, color_dict):
    return [color_dict[label] for label in labels]

In [ ]:
df_long_2.difficulty = df_long_2.difficulty.replace([1, 2, 3], ['Congruent', 'Incongruent', 'Implausible'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

order=['Congruent', 'Incongruent', 'Implausible']


fig, ax = plt.subplots(1, 1, figsize=(8, 8), sharey=True)

colors = [color_dict['crime'], color_dict['estate'], color_dict['finance']]
sns.set_palette(colors)
sns.swarmplot(x=x, y=y, data=df_long_2, hue=hue, alpha=.4, ax=ax, order=order)
sns.pointplot(x=x, y=y, data=df_long_2, hue=hue, dodge=0.2, join=None, ci=95, ax=ax, order=order)

box_pairs = [('Congruent', 'Incongruent'), ('Congruent', 'Implausible'), ('Incongruent', 'Implausible')]
p_values = [.0208, .00009, .0009]
add_stat_annotation(ax, data=df_long_2, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside', fontsize=16)

sns.despine()
ax.set_ylabel('Accuracy', fontsize=17)
ax.set_xlabel('Difficulty', fontsize=17)
#figure.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

handles, labels = ax.get_legend_handles_labels()
ax.legend([], [],frameon=False)

plt.setp(ax.get_yticklabels(), fontsize=15)
plt.setp(ax.get_xticklabels(), fontsize=15)
ax.set_title('Supplementary Experiment', fontsize=17, y=1.3)

ax.legend(handles[0:3], labels[0:3], labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), fontsize=17)

plt.tight_layout()
plt.subplots_adjust(wspace = 0.1)
plt.savefig('./plots/lmer_parts_exp2.pdf')

In [ ]:
df_long_2.difficulty = df_long_2.difficulty.replace([1, 2, 3], ['Congruent', 'Incongruent', 'Implausible'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

order=['Congruent', 'Incongruent', 'Implausible']


fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

### EXPERIMENT 2
df_long_3.difficulty = df_long_3.difficulty.replace([1, 2], ['Congruent', 'Incongruent'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

colors = [color_dict['crime'], color_dict['finance']]
sns.set_palette(colors)
sns.swarmplot(x=x, y=y, data=df_long_3, hue=hue, alpha=.4, ax=axs[0], order=order[:2])
sns.pointplot(x=x, y=y, data=df_long_3, hue=hue, join=None, dodge=0.1, ci=95, ax=axs[0], order=order[:2])

box_pairs = [('Congruent', 'Incongruent')]
p_values = [.0140]
#add_stat_annotation(ax=axs[0], data=df_long_3, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside', fontsize=16)
x1, x2 = 0, 1
y, h, col = 1.03, 0.03, 'k'
axs[0].plot([x1, x1, x2, x2], [y, y+h, y+h, y], color=col)
axs[0].text((x1 + x2)*0.5, y+1.2*h, '*', ha='center', va='bottom', color=col, fontsize=17)

sns.despine()
axs[0].set_ylabel('')
axs[0].set_xlabel('Difficulty', fontsize=17)
axs[0].legend([], [] ,frameon=False)
#fig.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

plt.setp(axs[0].get_yticklabels(), fontsize=15)
plt.setp(axs[0].get_xticklabels(), fontsize=15)
axs[0].set_title('Experiment 2', fontsize=17, y=1.1)

### EXPERIMETN 3
df_long_4.difficulty = df_long_4.difficulty.replace([1, 2], ['Congruent', 'Incongruent'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

colors = [color_dict['crime'], color_dict['finance']]
sns.set_palette(colors)
sns.swarmplot(x=x, y=y, data=df_long_4, hue=hue, alpha=.4, ax=axs[1], order=order[:2])
sns.pointplot(x=x, y=y, data=df_long_4, hue=hue, join=None, dodge=0.1, ci=95, ax=axs[1], order=order[:2])

box_pairs = [('Congruent', 'Incongruent')]
p_values = [.0300]
x1, x2 = 0, 1
y, h, col = 1.03, 0.03, 'k'
axs[1].plot([x1, x1, x2, x2], [y, y+h, y+h, y], color=col)
axs[1].text((x1 + x2)*0.5, y+1.2*h, '*', ha='center', va='bottom', color=col, fontsize=17)

sns.despine()
axs[1].set_ylabel('')
axs[1].set_xlabel('Difficulty', fontsize=17)
#fig.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

handles, labels = axs[1].get_legend_handles_labels()

plt.setp(axs[1].get_yticklabels(), fontsize=15)
plt.setp(axs[1].get_xticklabels(), fontsize=15)
axs[1].set_title('Experiment 3', fontsize=17, y=1.1)

axs[1].legend(handles[0:2], labels[0:2], labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), fontsize=17)

plt.tight_layout()
plt.subplots_adjust(wspace = 0.1)
plt.savefig('./plots/lmer_parts.pdf')

## Hamming distance

In [ ]:
df_ee_2 = df_long_2_ham
df_ee_2.difficulty = df_ee_2.difficulty.replace([1, 2, 3], ['Congruent', 'Incongruent', 'Implausible'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

fig, axs = plt.subplots(1, 3, figsize=(17, 8), sharey=True)

colors = [color_dict['crime'], color_dict['estate'], color_dict['finance']]
sns.set_palette(colors)
sns.barplot(x=x, y=y, data=df_ee_2, hue=hue, ax=axs[0])

box_pairs = [('Congruent', 'Incongruent'), ('Congruent', 'Implausible'), ('Incongruent', 'Implausible')]
p_values = [.0021, .00009, .1110]
add_stat_annotation(axs[0], data=df_ee_2, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside')

sns.despine()
axs[0].set_ylabel('Accuracy', fontsize=17)
axs[0].set_xlabel('Difficulty', fontsize=17)
#figure.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend([], [],frameon=False)
axs[0].set_title('A. Experiment 2', fontsize=17, y=1.3)

plt.setp(axs[0].get_yticklabels(), fontsize=15)
plt.setp(axs[0].get_xticklabels(), fontsize=15)


### EXPERIMETN 3
df_ee_3 = df_long_3_ham
df_ee_3.difficulty = df_ee_3.difficulty.replace([1, 2], ['Congruent', 'Incongruent'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

colors = [color_dict['crime'], color_dict['finance']]
sns.set_palette(colors)
sns.barplot(x=x, y=y, data=df_ee_3, hue=hue, ax=axs[1], order=['Congruent', 'Incongruent'])
box_pairs = [('Congruent', 'Incongruent')]
p_values = [.02]
add_stat_annotation(ax=axs[1], data=df_ee_3, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside')

sns.despine()
axs[1].set_ylabel('')
axs[1].set_xlabel('Difficulty', fontsize=17)
#fig.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

#handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend([], frameon=False)

plt.setp(axs[1].get_yticklabels(), fontsize=15)
plt.setp(axs[1].get_xticklabels(), fontsize=15)
axs[1].set_title('B. Experiment 3', fontsize=17, y=1.3)


#Experiment 4
df_long_4_ham.difficulty = df_long_4_ham.difficulty.replace([1, 2], ['Congruent', 'Incongruent'])
x = 'difficulty'
y = 'accuracy'
hue = 'scenario'

colors = [color_dict['crime'], color_dict['finance']]
sns.set_palette(colors)
sns.barplot(x=x, y=y, data=df_long_4_ham, hue=hue, ax=axs[2], order=['Congruent', 'Incongruent'])

box_pairs = [('Congruent', 'Incongruent')]
p_values = [.2084]
add_stat_annotation(ax=axs[2], data=df_long_4_ham, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside', fontsize=16)

sns.despine()
axs[2].set_ylabel('')
axs[2].set_xlabel('Difficulty', fontsize=17)
#fig.suptitle('Difference in accuracy between difficulty conditions', fontsize=16)

#handles, labels = axs[1].get_legend_handles_labels()
axs[2].legend(handles[0:3], labels[0:3], labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), fontsize=17)

plt.setp(axs[2].get_yticklabels(), fontsize=15)
plt.setp(axs[2].get_xticklabels(), fontsize=15)
axs[2].set_title('C. Experiment 4', fontsize=17, y=1.3)

plt.tight_layout()
plt.subplots_adjust(wspace = 0.1)
plt.savefig('./plots/lmer_parts_hamming.pdf')

# Priors 

## Same models with prior shape

In [ ]:

x = 'difficulty'
y = 'accuracy'
hue = 'lc_prior_bf'
prior_geq0 = [1, 0]
titles_priors = ['Prior best fit', 'No prior best fit']
prior_types = ['lc_prior_bf']
model_names = ['Adaptive Selective LC']
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
letter = ['A', 'B']

p_values_arr = [(.2061, .0092), (.3411, .0047)]
experiments = ['experiment_3', 'experiment_4']
for j, exp in enumerate(experiments):
    if j == 0:
        df_plot = df_long_3
    else:
        df_plot = df_long_4

    df_plot.lc_prior_bf.replace({0:'Flat', 1:'Non-flat'}, inplace=True)
    df_plot.difficulty.replace({1:'Congruent', 2:'Incongruent'}, inplace=True)
    order = ['Congruent', 'Incongruent']
    hue_order = ['Flat', 'Non-flat']
    sns.stripplot(x=x, y=y, data=df_plot, hue=hue, alpha=.4, ax=axs[j], dodge=True, order=order, hue_order=hue_order)
    sns.pointplot(x=x, y=y, data=df_plot, hue=hue, dodge=0.38, ci=95, ax=axs[j], order=order, hue_order=hue_order)

    box_pairs = [
        (('Congruent', 'Flat'), ('Incongruent', 'Flat')),
        (('Congruent', 'Non-flat'), ('Incongruent', 'Non-flat'))
    ]

    annotator = Annotator(axs[j], box_pairs, data=df_plot, x=x, y=y, hue=hue, order=order, hue_order=hue_order, verbose=False)
    annotator.configure(test_short_name="Tukey's range test", loc='outside', show_test_name=False, text_format='star', fontsize=16)
    annotator.set_pvalues(p_values_arr[j])
    annotator.annotate()
        
    #add_stat_annotation(axs[i, j], data=df_plot, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside')

    sns.despine()
    if j == 0:
        axs[j].set_ylabel('Accuracy', fontsize=17)
        axs[j].legend([], [], frameon=False)
    else:
        axs[j].set_ylabel('')
        axs[j].legend(handles[0:2], labels[0:2], labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), fontsize=17, title='Prior Shape', title_fontsize=17)
        
    if i == len(prior_types) - 1:
        axs[j].set_xlabel('Difficulty', fontsize=17)
    else:
        axs[j].set_xlabel('')

    plt.setp(axs[j].get_yticklabels(), fontsize=15)
    plt.setp(axs[j].get_xticklabels(), fontsize=15)

    handles, labels = axs[j].get_legend_handles_labels()
    #axs[j].legend(handles[0:2], labels[0:2])
    axs[j].set_title(f'{letter[j]}. Experiment {j+2}',  fontsize=17, y=1.25)

        

#fig.suptitle('Effect of difficulty for participants best fit with and without prior parameter', fontsize=18, y=1.01)


#fig.suptitle('Interaction of difficulty whether participants were best fit \nwith a flat or non-flat prior in predicting accuracy', fontsize=17)
plt.tight_layout()
plt.savefig('./plots/manip_effect_prior_bf_exp34.pdf')


In [ ]:
df_long_3.lc_prior_bf.value_counts(normalize=True)

In [ ]:
df_long_4.lc_prior_bf.value_counts(normalize=True)

In [ ]:
df_long_3.groupby(['lc_prior_bf', 'difficulty']).scenario.value_counts()

In [ ]:
df_long_3.groupby(['lc_prior_bf', 'difficulty']).scenario.value_counts()

In [ ]:
df_long_3.groupby(['lc_prior_bf', 'difficulty', 'scenario']).accuracy.mean()

# With Hamming distance

In [ ]:

x = 'difficulty'
y = 'hamming'
hue = 'lc_prior_bf'
prior_geq0 = [1, 0]
titles_priors = ['Prior best fit', 'No prior best fit']
prior_types = ['lc_prior_bf']
model_names = ['Adaptive Selective LC']
fig, axs = plt.subplots(1, 2, figsize=(12, 8), sharey=True)
letter = ['A', 'B']

p_values_arr = [(.1462, .0113), (.8436, .0012)]
experiments = ['experiment_3', 'experiment_4']
for j, exp in enumerate(experiments):
    if j == 0:
        df_plot = df_long_3
    else:
        df_plot = df_long_4

    df_plot.lc_prior_bf.replace({0:'Flat', 1:'Non-flat'}, inplace=True)
    df_plot.difficulty.replace({1:'Congruent', 2:'Incongruent'}, inplace=True)
    order = ['Congruent', 'Incongruent']
    hue_order = ['Flat', 'Non-flat']
    sns.stripplot(x=x, y=y, data=df_plot, hue=hue, alpha=.4, ax=axs[j], dodge=True, order=order, hue_order=hue_order)
    sns.pointplot(x=x, y=y, data=df_plot, hue=hue, dodge=0.38, ci=95, ax=axs[j], order=order, hue_order=hue_order)

    box_pairs = [
        (('Congruent', 'Flat'), ('Incongruent', 'Flat')),
        (('Congruent', 'Non-flat'), ('Incongruent', 'Non-flat'))
    ]

    annotator = Annotator(axs[j], box_pairs, data=df_plot, x=x, y=y, hue=hue, order=order, hue_order=hue_order, verbose=False)
    annotator.configure(test_short_name="Tukey's range test", loc='outside', show_test_name=False, text_format='star', fontsize=16)
    annotator.set_pvalues(p_values_arr[j])
    annotator.annotate()
        
    #add_stat_annotation(axs[i, j], data=df_plot, x=x, y=y, box_pairs=box_pairs, perform_stat_test=False, test_short_name="Tukey's range test", pvalues=p_values, verbose=2, loc='outside')

    sns.despine()
    if j == 0:
        axs[j].set_ylabel('Accuracy', fontsize=17)
        axs[j].legend([], [], frameon=False)
    else:
        axs[j].set_ylabel('')
        axs[j].legend(handles[0:2], labels[0:2], labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), fontsize=17, title='Prior Shape', title_fontsize=17)
        
    if i == len(prior_types) - 1:
        axs[j].set_xlabel('Difficulty', fontsize=17)
    else:
        axs[j].set_xlabel('')

    plt.setp(axs[j].get_yticklabels(), fontsize=15)
    plt.setp(axs[j].get_xticklabels(), fontsize=15)

    handles, labels = axs[j].get_legend_handles_labels()
    #axs[j].legend(handles[0:2], labels[0:2])
    axs[j].set_title(f'{letter[j]}. Experiment {j+2}',  fontsize=17, y=1.20)

        

#fig.suptitle('Effect of difficulty for participants best fit with and without prior parameter', fontsize=18, y=1.01)


#fig.suptitle('Interaction of difficulty whether participants were best fit \nwith a flat or non-flat prior in predicting accuracy', fontsize=17)
plt.tight_layout()
plt.savefig('./plots/manip_effect_prior_bf_exp34_hamming.pdf')


# Indirect links errors in labelled trials for experiment 3 and 4

Mixed effect model can be found in mixed_effect_models_euclidean_norm.R, last analysis

In [ ]:
trials_labelled = df_trials[df_trials.trial_name.isin(['crime', 'finance'])]

order = ['Congruent', 'Incongruent']
hue_order = ['Flat', 'Non-flat']

fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

trials_3_prior = trials_labelled[trials_labelled.experiment == 'experiment_3']
trials_3_prior.lc_prior_bf.replace({0:'Flat', 1:'Non-flat'}, inplace=True)
trials_3_prior.trial_type.replace({'congruent':'Congruent', 'incongruent':'Incongruent'}, inplace=True)

sns.stripplot(hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', data=trials_3_prior, ax=axs[0], alpha=0.2, dodge=0.2, hue_order=hue_order, order=order)
sns.pointplot(hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', data=trials_3_prior, ax=axs[0], dodge=0.38, ci=95, hue_order=hue_order, order=order)

box_pairs = [
    (('Flat', 'Congruent'), ('Non-flat', 'Congruent')),
    (('Flat', 'Incongruent'), ('Non-flat', 'Incongruent')),
]
box_pairs = [
    (('Congruent', 'Flat'), ('Incongruent', 'Flat')),
    (('Congruent', 'Non-flat'), ('Incongruent', 'Non-flat')),
]
p_values = [0.1143, 0.0080]

#annotator = Annotator(axs[0], box_pairs, data=trials_3_prior, hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', order=order, hue_order=hue_order, verbose=False)
#annotator.configure(test_short_name="Tukey's range test", loc='outside', show_test_name=False, text_format='star', fontsize=16)
#annotator.set_pvalues(p_values)
#annotator.annotate()

handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend([], [], frameon=False, fontsize=17)
axs[0].set_title('A. Experiment 2', fontsize=17, y=1.25)
axs[0].set_ylabel('Number of errors on indirect links',  fontsize=17)
axs[0].set_xlabel('',  fontsize=17)
axs[0].set_xticklabels(order,  fontsize=15)
axs[0].set_yticklabels(axs[0].get_yticklabels(), fontsize=15)


trials_4_prior = trials_labelled[trials_labelled.experiment == 'experiment_4']
trials_4_prior.lc_prior_bf.replace({0:'Flat', 1:'Non-flat'}, inplace=True)
trials_4_prior.trial_type.replace({'congruent':'Congruent', 'incongruent':'Incongruent'}, inplace=True)

sns.stripplot(hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', data=trials_4_prior, ax=axs[1], alpha=0.2, dodge=0.2, hue_order=hue_order, order=order)
sns.pointplot(hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', data=trials_4_prior, ax=axs[1], dodge=0.38, ci=95, hue_order=hue_order, order=order)


p_values = [0.4514, 0.0005]

annotator = Annotator(axs[1], box_pairs, data=trials_4_prior, hue='lc_prior_bf', y='num_indirect_errors', x='trial_type', order=order, hue_order=hue_order, verbose=False)
annotator.configure(test_short_name="Tukey's range test", loc='outside', show_test_name=False, text_format='star', fontsize=16)
annotator.set_pvalues(p_values)
annotator.annotate()

handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles[0:2], labels[0:2], fontsize=17, labelspacing=1, loc=6, bbox_to_anchor=(1, 0.5), title='Prior Shape', title_fontsize=17)
axs[1].set_title('B. Experiment 3', fontsize=17, y=1.25)
axs[1].set_ylabel('',  fontsize=17)
axs[1].set_xlabel('',  fontsize=17)
axs[1].set_xticklabels(order,  fontsize=15)
#axs[1].set_yticklabels(axs[1].get_yticklabels(), fontsize=15)

#fig.suptitle('Number of errors on indirect links for participants best fit with and without a non-flat prior parameter', fontsize=17)
sns.despine()
plt.tight_layout()
plt.savefig('./plots/indirect_links_errors_34.pdf')
plt.show()

# Root node frequencies

Aggregate interventional data

In [ ]:
# IMport fresh datasets
df_long_3 = pd.read_csv('./data/accuracy_lf_exp3_wprior.csv')
df_long_4 = pd.read_csv('./data/accuracy_lf_exp4_wprior.csv')

trial_types = interventions.groupby('utid').trial_type.unique().to_list()
trial_types = np.concatenate(trial_types).flatten()

utids = interventions.utid.unique()
print(utids.size)
df_inters = pd.DataFrame()
new_vars_dict = {}
new_vars = np.empty((utids.size, 50))
new_vars[:] = np.nan
var_order = np.empty((utids.size, 50))
var_order[:] = np.nan

var_pos = np.empty((utids.size, 50))
var_pos[:] = np.nan

var_name_pos = np.empty((utids.size, 50)).astype(str)
var_root_pos = np.empty((utids.size, 50)).astype(str)
var_gtroot_pos = np.empty((utids.size, 50)).astype(str)

nums = []
for i, unique_trial in enumerate(utids):
    df_chunk = interventions[interventions.utid == unique_trial].sort_values('int_num')
    new_var = []
    intervened_var = []
    var_positions = []
    for j, inter in enumerate(df_chunk.index):
        var_touched = df_chunk.loc[inter, 'variable']  
        var_prior_type = df_chunk.loc[inter, 'prior_type']
        var_gt_type = df_chunk.loc[inter, 'gt_type']

        if var_touched not in intervened_var:
            intervened_var.append(var_touched)
            new_var.append(1)
            new_vars[i, j] = 1
        else:
            new_var.append(0)
            new_vars[i, j] = 0

        
        var_place = df_chunk.loc[inter, 'var_position']
        var_pos[i, j] = var_place + 1
        var_name_pos[i, j] = var_touched
        var_root_pos[i, j] = var_prior_type
        var_gtroot_pos[i, j] = var_gt_type
        if not var_positions:
            if var_place == 0:
                var_order[i, j] = 1
            else:
                var_order[i, j] = 0
            
        elif var_place >= var_positions[-1]:
            var_order[i, j] = 1
        else:
            var_order[i, j] = 0
            
        var_positions.append(var_place)
    
    nums.append(len(new_var))
    
    new_vars_dict[unique_trial] = new_var


experiments = [int(utid[0]) for utid in utids]
pids = [utid.split('-')[1] for utid in utids]

prior_bf = np.zeros(len(pids))
for i, pid in enumerate(pids):
    if pid in df_long_3.participant.to_list():
        prior_bf[i] = df_long_3.loc[df_long_3.participant == pid, 'lc_prior_bf'].to_list()[0]
    elif pid in df_long_4.participant.to_list():
        prior_bf[i] = df_long_4.loc[df_long_4.participant == pid, 'lc_prior_bf'].to_list()[0]
    else:
        prior_bf[i] = np.nan


df_order = pd.DataFrame(columns=['experiment', 'utid', 'pid', 'prior_bf', 'trial_type'] + [f'inter_{i}' for i in range(35)])
df_order['experiment'] = experiments
df_order['utid'] = utids
df_order['pid'] = pids
df_order['prior_bf'] = prior_bf
df_order['trial_type'] = trial_types

df_order_name = pd.DataFrame(columns=['experiment', 'utid', 'pid', 'prior_bf', 'trial_type'] + [f'inter_{i}' for i in range(35)])
df_order_name['experiment'] = experiments
df_order_name['utid'] = utids
df_order_name['pid'] = pids
df_order_name['prior_bf'] = prior_bf
df_order_name['trial_type'] = trial_types

df_order_root = pd.DataFrame(columns=['experiment', 'utid', 'pid', 'prior_bf', 'trial_type'] + [f'inter_{i}' for i in range(35)])
df_order_root['experiment'] = experiments
df_order_root['utid'] = utids
df_order_root['pid'] = pids
df_order_root['prior_bf'] = prior_bf
df_order_root['trial_type'] = trial_types

df_order_gtroot = pd.DataFrame(columns=['experiment', 'utid', 'pid', 'prior_bf', 'trial_type'] + [f'inter_{i}' for i in range(35)])
df_order_gtroot['experiment'] = experiments
df_order_gtroot['utid'] = utids
df_order_gtroot['pid'] = pids
df_order_gtroot['prior_bf'] = prior_bf
df_order_gtroot['trial_type'] = trial_types

for i, col in enumerate([f'inter_{i}' for i in range(35)]):
    df_order.loc[df_order.index, col ] = var_pos[:, i]
    df_order_name.loc[df_order_name.index, col ] = var_name_pos[:, i]
    df_order_root.loc[df_order_root.index, col] = var_root_pos[:, i]
    df_order_gtroot.loc[df_order_gtroot.index, col] = var_gtroot_pos[:, i]
    #print(var_root_pos[:, i])

df_order_new = pd.DataFrame(columns=['experiment', 'utid', 'pid', 'prior_bf', 'trial_type'] + [f'inter_{i}' for i in range(35)])

df_order_new['experiment'] = experiments
df_order_new['utid'] = utids
df_order_new['pid'] = pids
df_order_new['prior_bf'] = prior_bf
df_order_new['trial_type'] = trial_types
for i, col in enumerate([f'inter_{i}' for i in range(35)]):
    df_order_new.loc[df_order_new.index, col ] = new_vars[:, i]


label_trials = ['label', 'congruent', 'incongruent', 'implausible']
df_order.trial_type.unique()
generic_trials = ['generic_0', 'generic_1', 'generic_2', 'generic_3']

df_order_generic = df_order[df_order.trial_type.isin(generic_trials)]
df_order_labels = df_order[df_order.trial_type.isin(label_trials)]

df_order_new_generic = df_order_new[df_order_new.trial_type.isin(generic_trials)]
df_order_new_labels = df_order_new[df_order_new.trial_type.isin(label_trials)]

df_c = df_order[df_order.trial_type == label_trials[1]]
df_congruent_3 = df_c[df_c.experiment == 3]
df_i =  df_order[df_order.trial_type == label_trials[2]]
df_incongruent_3 = df_i[df_i.experiment == 3]


idx_inter = 15
K = 3
count_generic = np.zeros((K, idx_inter))
count_label = np.zeros((K, idx_inter))

count_variable = np.zeros((2, K, idx_inter))

count_var_exp3 = np.zeros((2, K, idx_inter))


for i, col in enumerate(df_order_generic.columns[3:3+idx_inter]):
    for k in range(K):
        count_generic[k, i] = df_order_generic[df_order_generic[f'inter_{i}'] == k+1].shape[0]
        count_label[k, i] = df_order_labels[df_order_labels[f'inter_{i}'] == k+1].shape[0]

        count_variable[0, k, i] = df_order_generic[df_order_generic[f'inter_{i}'] == k+1].shape[0]
        count_variable[1, k, i] = df_order_labels[df_order_labels[f'inter_{i}'] == k+1].shape[0]

        count_var_exp3[0, k, i] = df_congruent_3[df_congruent_3[f'inter_{i}'] == k+1].shape[0]
        count_var_exp3[1, k, i] = df_incongruent_3[df_incongruent_3[f'inter_{i}'] == k+1].shape[0]





In [ ]:
new_df = df_order_gtroot[(df_order_gtroot == 'root') | (df_order_gtroot == 'leaf')]
new_df[['experiment', 'utid', 'pid', 'prior_bf', 'trial_type']] = df_order_root[['experiment', 'utid', 'pid', 'prior_bf', 'trial_type']]

df_root_counts = new_df[new_df.experiment.isin([3, 4])].replace({'nan':np.nan})

df_root_counts.prior_bf.replace({0:'flat', 1:'non flat'}, inplace=True)

prior_bfs = ['non flat']
trials = ['congruent', 'incongruent']
experiment_idx = [3, 4]

loc_idx_inter = 7

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(14, 7), sharex=True)

hue_order = ['root', 'leaf']

for i, trial in enumerate(trials):
    df_root_counts_trial = df_root_counts[df_root_counts.trial_type == trial]

    df_root_trial_prior = df_root_counts_trial[df_root_counts_trial.prior_bf == prior_bfs[0]]

    for j, exp_idx in enumerate(experiment_idx):
        #print(exp_idx)
        df_root_trial_prior_exp = df_root_trial_prior[df_root_trial_prior.experiment == exp_idx]
        #print(df_root_trial_prior_exp.shape)
        
        #print(df_root_trial_prior.shape)

        mapper = {k:v for k, v in zip(df_root_counts_trial.columns[5:loc_idx_inter+5], np.arange(df_root_counts_trial.columns[5:loc_idx_inter+5].size))}
        df_counts_plot_c = df_root_trial_prior_exp[df_root_trial_prior_exp.columns[5:loc_idx_inter + 5]].rename(mapper, axis=1)
        df_counts_plot = df_counts_plot_c.melt()

        
        rows = ['root', 'leaf']
        df_final = pd.DataFrame(index=rows, columns=df_counts_plot_c.columns)
        for col in df_counts_plot_c.columns:
            counts = df_counts_plot_c[col].value_counts()

            #print(counts)
            #print(counts.loc['root'])
            for r in rows:
                if r in counts.index:
                    df_final.loc[r, col] = counts.loc[r]


        df_final.transpose().plot(kind='bar', stacked=True, ax=axs[i, j])


        #sns.countplot(x='variable', hue='value', data=df_counts_plot.dropna(), hue_order=hue_order, ax=axs[i, j])
        #df_counts_plot.plot(kind='bar', stacked=True, ax=axs[i, j])
        if i == 0:
            axs[i, j].set_title(f'Experiment {exp_idx}, Prior shape: {prior_bfs[0]} \n\n Trial: {trial.capitalize()}', fontsize=19)
        else:
            axs[i, j].set_title(f'Trial: {trial.capitalize()}', fontsize=19)
            axs[i, j].set_xlabel('Index of intervention', fontsize=17)

        handles, labels = axs[i,j].get_legend_handles_labels()
        if j == 0:
            axs[i, j].set_ylabel('Intervention counts', fontsize=17)
            axs[i, j].legend([], [],frameon=False)

        if i == 0:
            axs[i, j].legend([], [],frameon=False)
            
        plt.setp(axs[i, j].get_yticklabels(), fontsize=15)
        plt.setp(axs[i, j].get_xticklabels(), fontsize=15, rotation=0)

        

sns.despine()
        

plt.tight_layout()

axs[1, 1].legend(handles[0:2], labels[0:2], labelspacing=1, loc=6, bbox_to_anchor=(1, 1.2), fontsize=17)
plt.savefig('./plots/interventions_roots.pdf')
plt.show()